### Importing dependencies

In [1]:
import pandas as pd
import numpy as np
import ast
import nltk
from nltk.corpus import stopwords
import difflib
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Exploring the dataset

In [3]:
movies_df = pd.read_csv('datasets/tmdb_5000_movies.csv')
credits_df = pd.read_csv("datasets/tmdb_5000_credits.csv")

In [4]:
movies_df.shape

(4803, 20)

In [5]:
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [7]:
vote_count_df = movies_df[['id', 'title', 'vote_count']]

### Preprocessing the data

In [8]:
movies_df = movies_df[['id', 'title', 'genres', 'keywords', 'overview', 'tagline', 'release_date']]

In [9]:
movies_df.head(3)

,id,title,genres,keywords,overview,tagline,release_date
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,2009-12-10
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",2007-05-19
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,2015-10-26


In [10]:
credits_df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [11]:
credits_df.rename(columns={'movie_id':'id'}, inplace=True)

In [12]:
credits_df.drop(columns='title', inplace=True)

In [13]:
new_df = movies_df.merge(credits_df, on='id')

In [14]:
new_df.head(3)

,id,title,genres,keywords,overview,tagline,release_date,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,2009-12-10,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",2007-05-19,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,2015-10-26,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [15]:
new_df['title'].nunique()

4800

In [16]:
new_df['release_date'] = new_df['release_date'].apply(lambda x:str(x))
new_df['release_date'] = new_df['release_date'].apply(lambda x:x[0:4])

In [17]:
new_df.head(3)

,id,title,genres,keywords,overview,tagline,release_date,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,2009,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",2007,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,2015,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [18]:
new_df.head(10)

,id,title,genres,keywords,overview,tagline,release_date,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,2009,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",2007,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,2015,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Legend Ends,2012,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","Lost in our world, found in another.",2012,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,559,Spider-Man 3,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...","[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",The seemingly invincible Spider-Man goes up ag...,The battle within.,2007,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."
6,38757,Tangled,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",When the kingdom's most wanted-and most charmi...,They're taking adventure to new lengths.,2010,"[{""cast_id"": 34, ""character"": ""Flynn Rider (vo...","[{""credit_id"": ""52fe46db9251416c91062101"", ""de..."
7,99861,Avengers: Age of Ultron,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 8828, ""name"": ""marvel comic""}, {""id"": ...",When Tony Stark tries to jumpstart a dormant p...,A New Age Has Come.,2015,"[{""cast_id"": 76, ""character"": ""Tony Stark / Ir...","[{""credit_id"": ""55d5f7d4c3a3683e7e0016eb"", ""de..."
8,767,Harry Potter and the Half-Blood Prince,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 616, ""name"": ""witch""}, {""id"": 2343, ""n...","As Harry begins his sixth year at Hogwarts, he...",Dark Secrets Revealed,2009,"[{""cast_id"": 3, ""character"": ""Harry Potter"", ""...","[{""credit_id"": ""52fe4273c3a36847f801fab1"", ""de..."
9,209112,Batman v Superman: Dawn of Justice,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 7002...",Fearing the actions of a god-like Super Hero l...,Justice or revenge,2016,"[{""cast_id"": 18, ""character"": ""Bruce Wayne / B...","[{""credit_id"": ""553bf23692514135c8002886"", ""de..."


In [19]:
new_df.isnull().sum()

id                0
title             0
genres            0
keywords          0
overview          3
tagline         844
release_date      0
cast              0
crew              0
dtype: int64

In [20]:
new_df.fillna('', inplace=True)

### Preprocessing Functions

In [21]:
def changeDuplicateTitle():
    title_list = new_df['title'].copy()
    unique = set()
    duplicate_titles = [title for title in new_df['title'] if title in unique or unique.add(title)]
    for i in range(0, len(new_df['title'])):
        if new_df['title'][i] in duplicate_titles:
            title_list[i] = new_df['title'][i] + " " + "(" + new_df['release_date'][i] + ")"
    return title_list

In [22]:
def getWord(dict_list):
    name_list = []
    for item in ast.literal_eval(dict_list):
        name_list.append(item['name'])
    return name_list

In [23]:
def getCast(dict_list):
    cast_list = []
    count = 0
    for item in ast.literal_eval(dict_list):
        if count != 10:
            cast_list.append(item['name'])
            count += 1
    return cast_list

In [24]:
def getDirector(dict_list):
    director_list = []
    for item in ast.literal_eval(dict_list):
        if item['job']=="Director":
            director_list.append(item['name'])
    return director_list

In [25]:
def removeStopwords(overviews):
    stopwords_english = set(stopwords.words('english'))
    new_overviews = [overview for overview in overviews if overview not in stopwords_english]
    return new_overviews

In [26]:
new_df['genres'] = new_df['genres'].apply(getWord)
new_df['keywords'] = new_df['keywords'].apply(getWord)

In [27]:
new_df['cast'] = new_df['cast'].apply(getCast)

In [28]:
new_df['crew'] = new_df['crew'].apply(getDirector)

In [29]:
new_df['title'] = changeDuplicateTitle()

In [30]:
new_df['overview'] = new_df['overview'].apply(lambda x:x.lower())
new_df['tagline'] = new_df['tagline'].apply(lambda x:x.lower())

In [31]:
new_df['overview'] = new_df['overview'].apply(lambda x:str(x))
new_df['overview'] = new_df['overview'].apply(lambda x:x.split())
new_df['overview'] = new_df['overview'].apply(removeStopwords)

In [32]:
new_df['tagline'] = new_df['tagline'].apply(lambda x:str(x))
new_df['tagline'] = new_df['tagline'].apply(lambda x:x.split())
new_df['tagline'] = new_df['tagline'].apply(removeStopwords)

In [33]:
new_df.rename(columns={'crew':'director'}, inplace=True)

In [34]:
new_df.head(3)

,id,title,genres,keywords,overview,tagline,release_date,cast,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[22nd, century,, paraplegic, marine, dispatche...","[enter, world, pandora.]",2009,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[captain, barbossa,, long, believed, dead,, co...","[end, world,, adventure, begins.]",2007,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[cryptic, message, bond’s, past, sends, trail,...","[plan, one, escapes]",2015,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]


In [35]:
new_df['tags'] = new_df['director'] + new_df['overview'] + new_df['genres'] + new_df['keywords'] + new_df['tagline'] + new_df['cast']

In [36]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [37]:
new_df.head(3)

,id,title,genres,keywords,overview,tagline,release_date,cast,director,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[22nd, century,, paraplegic, marine, dispatche...","[enter, world, pandora.]",2009,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],"james cameron 22nd century, paraplegic marine ..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[captain, barbossa,, long, believed, dead,, co...","[end, world,, adventure, begins.]",2007,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski],"gore verbinski captain barbossa, long believed..."
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[cryptic, message, bond’s, past, sends, trail,...","[plan, one, escapes]",2015,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes],sam mendes cryptic message bond’s past sends t...


In [38]:
movies = new_df[['id', 'title', 'tags']]

In [39]:
movies.head(10)

,id,title,tags
0,19995,Avatar,"james cameron 22nd century, paraplegic marine ..."
1,285,Pirates of the Caribbean: At World's End,"gore verbinski captain barbossa, long believed..."
2,206647,Spectre,sam mendes cryptic message bond’s past sends t...
3,49026,The Dark Knight Rises,christopher nolan following death district att...
4,49529,John Carter,"andrew stanton john carter war-weary, former m..."
5,559,Spider-Man 3,sam raimi seemingly invincible spider-man goes...
6,38757,Tangled,byron howard nathan greno kingdom's wanted-and...
7,99861,Avengers: Age of Ultron,joss whedon tony stark tries jumpstart dormant...
8,767,Harry Potter and the Half-Blood Prince,"david yates harry begins sixth year hogwarts, ..."
9,209112,Batman v Superman: Dawn of Justice,zack snyder fearing actions god-like super her...


### Building the model

In [40]:
vectorizer = TfidfVectorizer()

In [41]:
feature_vectors = vectorizer.fit_transform(movies['tags'])

In [42]:
print(feature_vectors)

  (0, 1242)	0.12018031916049918
  (0, 19400)	0.1330327469031731
  (0, 32384)	0.13498635567833853
  (0, 36317)	0.10556063731446619
  (0, 26347)	0.145885174645847
  (0, 5815)	0.145885174645847
  (0, 22762)	0.08649942167673325
  (0, 8471)	0.05246526064021421
  (0, 17568)	0.09229151487244029
  (0, 28345)	0.11758883452933991
  (0, 13683)	0.1153155258728684
  (0, 28653)	0.10687003769209964
  (0, 22135)	0.08665078687459217
  (0, 19188)	0.1184130650571401
  (0, 32063)	0.07473345476588938
  (0, 36200)	0.104736406786666
  (0, 30781)	0.1132907379939111
  (0, 29245)	0.1184130650571401
  (0, 37403)	0.10433763585262244
  (0, 36853)	0.13498635567833853
  (0, 29299)	0.0758026988108711
  (0, 36827)	0.053126238363414194
  (0, 11038)	0.10732789141782527
  (0, 276)	0.08997072313299644
  (0, 31484)	0.10076079027105478
  :	:
  (4802, 31151)	0.06433815423970003
  (4802, 4628)	0.09468727204724658
  (4802, 21401)	0.0663343293533037
  (4802, 8448)	0.08920031812747566
  (4802, 17624)	0.06781938715028611
  (4802,

### Calculating the similarity score

In [43]:
similarity = cosine_similarity(feature_vectors)

In [44]:
print(similarity)

[[1.         0.01894931 0.01149673 ... 0.00777261 0.0143155  0.        ]
 [0.01894931 1.         0.01767354 ... 0.01326887 0.01383494 0.00749543]
 [0.01149673 0.01767354 1.         ... 0.01172655 0.02727491 0.00231376]
 ...
 [0.00777261 0.01326887 0.01172655 ... 1.         0.01221744 0.02216497]
 [0.0143155  0.01383494 0.02727491 ... 0.01221744 1.         0.02004876]
 [0.         0.00749543 0.00231376 ... 0.02216497 0.02004876 1.        ]]


### Making recommendations

In [45]:
def recommend(movie):
    movie_list = movies['title'].tolist()
    movie_match = difflib.get_close_matches(movie, movie_list)
    closest_match = movie_match[0]
    movie_index = movies[movies['title'] == closest_match].index[0]
    distances = similarity[movie_index]
    recommended_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:21]
    
    for i in recommended_list:
        print(movies.iloc[i[0]].title)

In [46]:
try:
    search_movie = input("Search for movies similar to >> ")
    recommend(search_movie)
except IndexError:
    print("No results found")

Search for movies similar to >> The Dark Knight
The Dark Knight Rises
Batman Begins
Batman Returns
Batman Forever
Batman: The Dark Knight Returns, Part 2
Batman (1989)
Batman v Superman: Dawn of Justice
Batman & Robin
Law Abiding Citizen
Defendor
Kick-Ass
Superman
Sherlock Holmes: A Game of Shadows
Man of Steel
Slow Burn
Secret in Their Eyes
The Lego Movie
Superman II
Superman III
The Pledge


### Saving model

In [47]:
vote_count_df.to_pickle("model/vote_count_df.pkl")
movies.to_pickle("model/movies.pkl")

In [48]:
pickle.dump(similarity, open("model/similarity.pkl", "wb"))